# Autor: Victor Venites
- Data: 19/09/2023

# Dependencias

In [1]:
import pandas as pd
import random

# Definindo Cenário

In [2]:
posicao_inicial = 1
posicao_final = 36
numero_simulacoes = 10000

In [3]:
dict_posicoes_cobras = {3: 16, 5: 7, 15: 25, 18: 20, 21: 32}

In [4]:
dict_posicoes_escadas = {12: 2, 14: 11, 17: 4, 31: 19, 35: 22}

# Simulação

In [5]:
def rolar_dado():
    return random.randint(1, 6)

In [6]:
def turno(posicao_atual, contador_cobras, contador_dados_jogados, imunidade_jogador_B):
    #print(f"posicao_atual -> {posicao_atual}")
    jogada_dado = rolar_dado()
    contador_dados_jogados += 1
    #print(f"jogada_dado -> {jogada_dado}")
    posicao_atual += jogada_dado
    #print(f"posicao_atual -> {posicao_atual}")

    if posicao_atual in dict_posicoes_cobras:
        if not imunidade_jogador_B:
            posicao_atual = dict_posicoes_cobras[posicao_atual]
            contador_cobras += 1
            #print(f"contador_cobras -> {contador_cobras}")
        else:
            imunidade_jogador_B = False
    if posicao_atual in dict_posicoes_escadas:
        posicao_atual = dict_posicoes_escadas[posicao_atual]
        #print(f"posicao_atual -> {posicao_atual}")
    return posicao_atual, contador_cobras, contador_dados_jogados, imunidade_jogador_B

In [7]:
def jogadas(posicao_a, posicao_b, imunidade_jogador_B):
    jogador_atual = "A"
    contador_cobras = 0
    contador_dados_jogados = 0
    while posicao_a < posicao_final and posicao_b < posicao_final:
        if jogador_atual == "A":
            posicao_a, contador_cobras, contador_dados_jogados, imunidade_jogador_B = turno(posicao_a, contador_cobras, contador_dados_jogados, imunidade_jogador_B)
        else:
            posicao_b, contador_cobras, contador_dados_jogados, imunidade_jogador_B = turno(posicao_b, contador_cobras, contador_dados_jogados, imunidade_jogador_B)
        
        if jogador_atual == "A":
            jogador_atual = "B"
        else:
            jogador_atual = "A"
    vencedor = "A" if posicao_a >= posicao_final else "B"
    return vencedor, contador_cobras, contador_dados_jogados

In [8]:
def simulacao(quantidade_simulacoes, posicao_a = posicao_inicial, posicao_b = posicao_inicial, imunidade_jogador_B = False):
    # print(f"posicao_b -> {posicao_b}")
    resultados = []
    for _ in range(quantidade_simulacoes):
        vencedor = jogadas(posicao_a, posicao_b, imunidade_jogador_B)
        resultados.append(vencedor)

    return resultados

In [9]:
resultados = simulacao(numero_simulacoes)
resultados = pd.DataFrame(resultados, columns=["Vencedor", "Contador_Cobras", "Contador_Dados_Jogados"])

# Question 1
- In a two person game, what is the probability that the player who starts the game wins?

In [10]:
resultados["Vencedor"].value_counts(normalize=True)

Vencedor
A    0.5209
B    0.4791
Name: proportion, dtype: float64

# Question 2
- On average, how many snakes are landed on in each game?

In [11]:
print(f"Na média cai em {resultados['Contador_Cobras'].mean()} cobras por jogo, ou {round(resultados['Contador_Cobras'].mean())} cobras arredondado.")

Na média cai em 2.5111 cobras por jogo, ou 3 cobras arredondado.


# Question 3
- If each time a player landed on a ladder and there was only a 50% chance they could take it, what is the average number of rolls needed to complete a game?

In [12]:
print(f"Na média se vence com {resultados['Contador_Dados_Jogados'].mean()} dados rolados por jogo, ou {round(resultados['Contador_Dados_Jogados'].mean())} dados arredondado.")

Na média se vence com 19.0267 dados rolados por jogo, ou 19 dados arredondado.


# Question 4
- Starting with the base game, you decide you want the game to have approximately fair odds. You do this by changing the square that Player 2 starts on. Which square for Player 2’s start position gives the closest to equal odds for both players?

In [13]:
resultados_vantagem_b = []
for i in range(10):
    resultados_questao_4 = simulacao(numero_simulacoes, posicao_b = posicao_inicial + i)
    resultados_questao_4 = pd.DataFrame(resultados_questao_4, columns=["Vencedor", "Contador_Cobras", "Contador_Dados_Jogados"])
    vencedor_teste = resultados_questao_4["Vencedor"].value_counts(normalize=True).index[0]
    Percentual_A = resultados_questao_4["Vencedor"].value_counts(normalize=True)[0]
    Percentual_B = resultados_questao_4["Vencedor"].value_counts(normalize=True)[1]
    resultados_vantagem_b.append([i + 1, vencedor_teste, Percentual_A, Percentual_B])
df_vantagem_b = pd.DataFrame(resultados_vantagem_b, columns=["Posicao_Inicial_B", "Vencedor", "Percentual_Vencedor", "Percentual_Perdedor"])
df_vantagem_b

,Posicao_Inicial_B,Vencedor,Percentual_Vencedor,Percentual_Perdedor
0,1,A,0.5296,0.4704
1,2,A,0.5237,0.4763
2,3,A,0.5409,0.4591
3,4,A,0.5274,0.4726
4,5,A,0.5225,0.4775
5,6,A,0.5162,0.4838
6,7,B,0.5027,0.4973
7,8,B,0.5163,0.4837
8,9,B,0.5523,0.4477
9,10,B,0.5804,0.4196


# Question 5
- In a different attempt to change the odds of the game, instead of starting Player 2 on a different square, you decide to give Player 2 immunity to the first snake that they land on. What is the approximate probability that Player 1 wins now?

In [14]:
resultados = simulacao(numero_simulacoes, imunidade_jogador_B = True)
resultados = pd.DataFrame(resultados, columns=["Vencedor", "Contador_Cobras", "Contador_Dados_Jogados"])
resultados["Vencedor"].value_counts(normalize=True)

Vencedor
A    0.5055
B    0.4945
Name: proportion, dtype: float64